In [1]:
import numpy as np
import pandas as pd
import joblib

In [2]:
dataframe=pd.read_csv("player-tas-regression-test.csv")
print(dataframe.shape)

(3534, 92)


In [3]:
len(dataframe)

3534

In [4]:
# drop nulll & unusfull columns 
nullcols=['id', 'name', 'full_name', 'birth_date','traits','national_team','national_rating','national_team_position','national_jersey_number','club_join_date','contract_end_year','tags','traits']
dataframe.drop(nullcols,axis=1,inplace=True)
dataframe.shape

(3534, 80)

In [5]:
c= dataframe['positions'].str.split(',').max()
c # max num of position can player play on it in data is 

['ST', 'RW', 'RM']

In [6]:
def spliPostion(data):
    max_values = np.max([
        data['positions'].apply(lambda x: len(str(x).split(','))).max(),
    ])
    return data['positions'].str.split(',').apply(lambda x: x + ['0'] * (max_values - len(x)))

In [7]:
# the lare number player can play in it is 3
play_postion = spliPostion(dataframe)
dataframe['position_first'] = play_postion.apply(lambda x: x[0])
dataframe['position_second'] = play_postion.apply(lambda x: x[1])
dataframe['position_third'] = play_postion.apply(lambda x: x[2])
dataframe.drop('positions', axis=1, inplace=True)

In [8]:
nationality=joblib.load('model/nationality_transform')
dic = dict(zip(nationality.classes_, nationality.transform(nationality.classes_)))
dataframe['nationality']=dataframe['nationality'].map(dic).fillna(-1).astype(int)


preferred_foot=joblib.load('model/preferred_foot_transform')
dic = dict(zip(preferred_foot.classes_, preferred_foot.transform(preferred_foot.classes_)))
dataframe['preferred_foot']=dataframe['preferred_foot'].map(dic).fillna(-1).astype(int)


body_type=joblib.load('model/body_type_transform')
dic = dict(zip(body_type.classes_, body_type.transform(body_type.classes_)))
dataframe['body_type']=dataframe['body_type'].map(dic).fillna(-1).astype(int)


club_team=joblib.load('model/club_team_transform')
dic = dict(zip(club_team.classes_, club_team.transform(club_team.classes_)))
dataframe['club_team']=dataframe['club_team'].map(dic).fillna(-1).astype(int)


club_position=joblib.load('model/club_position_transform')
dic = dict(zip(club_position.classes_, club_position.transform(club_position.classes_)))
dataframe['club_position']=dataframe['club_position'].map(dic).fillna(-1).astype(int)


position_first=joblib.load('model/position_first_transform')
dic = dict(zip(position_first.classes_, position_first.transform(position_first.classes_)))
dataframe['position_first']=dataframe['position_first'].map(dic).fillna(-1).astype(int)


position_second=joblib.load('model/position_second_transform')
dic = dict(zip(position_second.classes_, position_second.transform(position_second.classes_)))
dataframe['position_second']=dataframe['position_second'].map(dic).fillna(-1).astype(int)


position_third=joblib.load('model/position_third_transform')
dic = dict(zip(position_third.classes_, position_third.transform(position_third.classes_)))
dataframe['position_third']=dataframe['position_third'].map(dic).fillna(-1).astype(int)


In [9]:
dataframe.head(1)

,age,height_cm,weight_kgs,nationality,overall_rating,potential,wage,preferred_foot,international_reputation(1-5),weak_foot(1-5),...,RWB,LB,LCB,CB,RCB,RB,value,position_first,position_second,position_third
0,21,182.88,74.8,146,67,76,3000,1,1,3,...,64+2,63+2,60+2,60+2,60+2,63+2,1200000,4,3,10


In [10]:
dataframe['upper_work_rate'] = dataframe['work_rate'].apply(lambda x: x.split('/')[0])
dataframe['down_work_rate'] = dataframe['work_rate'].apply(lambda x: x.split('/')[1])


In [11]:
dataframe['height_cm'] = dataframe['height_cm'].apply(lambda x:x/100)

In [12]:
dataframe.head(3)

,age,height_cm,weight_kgs,nationality,overall_rating,potential,wage,preferred_foot,international_reputation(1-5),weak_foot(1-5),...,LCB,CB,RCB,RB,value,position_first,position_second,position_third,upper_work_rate,down_work_rate
0,21,1.8288,74.8,146,67,76,3000,1,1,3,...,60+2,60+2,60+2,63+2,1200000,4,3,10,Medium,Medium
1,18,1.5494,67.1,76,60,82,1000,1,1,3,...,58+2,58+2,58+2,59+2,400000,2,5,0,Medium,Medium
2,28,1.7526,64.9,129,63,63,1000,1,1,3,...,41+2,41+2,41+2,50+2,375000,0,11,5,Medium,Medium


In [13]:
release_clause_euro=joblib.load('model/minmax_release_clause_euro')
dataframe['release_clause_euro']=release_clause_euro.transform(dataframe[['release_clause_euro']])

wage=joblib.load('model/wage_release_clause_euro')
dataframe['wage']=wage.transform(dataframe[['wage']])


In [14]:
# rawan pre
columnsIter2 = pd.DataFrame(dataframe, columns=['LS','ST','RS','LW','LF','CF','RF','RW','LAM','CAM','RAM','LM','LCM','CM','RCM','RM','LWB','LDM','CDM','RDM','RWB','LB','LCB','CB','RCB','RB'],index=[dataframe.index])

for(columnName1, columnData1) in columnsIter2.iteritems():
    dataframe[columnName1] = dataframe[columnName1].str.replace('+', '.')
    dataframe[columnName1]= pd.to_numeric(dataframe[columnName1])

#for null numerical values:
columnsIter = pd.DataFrame(dataframe, columns=['LS','ST','RS','LW','LF','CF','RF','RW','LAM','CAM','RAM','LM','LCM','CM','RCM','RM','LWB','LDM','CDM','RDM','RWB','LB','LCB','CB','RCB','RB','wage','release_clause_euro','club_rating','club_jersey_number','value'],index=[dataframe.index])
for (columnName, columnData) in columnsIter.iteritems():
    dataframe[columnName] = dataframe[columnName].fillna(dataframe[columnName].mean())

C:\Users\EGYPT_~1\AppData\Local\Temp/ipykernel_5688/1838296454.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe[columnName1] = dataframe[columnName1].str.replace('+', '.')


In [15]:
dataframe.head()

,age,height_cm,weight_kgs,nationality,overall_rating,potential,wage,preferred_foot,international_reputation(1-5),weak_foot(1-5),...,LCB,CB,RCB,RB,value,position_first,position_second,position_third,upper_work_rate,down_work_rate
0,21,1.8288,74.8,146,67,76,0.004405,1,1,3,...,60.2,60.2,60.2,63.2,1200000,4,3,10,Medium,Medium
1,18,1.5494,67.1,76,60,82,0.000000,1,1,3,...,58.2,58.2,58.2,59.2,400000,2,5,0,Medium,Medium
2,28,1.7526,64.9,129,63,63,0.000000,1,1,3,...,41.2,41.2,41.2,50.2,375000,0,11,5,Medium,Medium
3,20,1.8542,74.8,28,51,61,0.002203,1,1,3,...,48.2,48.2,48.2,49.2,60000,11,0,0,Medium,Medium
4,25,1.8288,77.1,27,63,68,0.000000,1,1,3,...,37.2,37.2,37.2,38.2,500000,14,0,0,Medium,Medium


In [16]:
def convert_rate(col):
    coll =joblib.load('model/'+col+'_predect')
    dic = dict(zip(coll.classes_, coll.transform(coll.classes_)))
    dataframe[col]=dataframe[col].map(dic).fillna(-1).astype(int)
    dataframe[col]=dataframe[col].apply(lambda x:x+1)

convert_rate('upper_work_rate')
convert_rate('down_work_rate')

In [17]:
dataframe.head(1)

,age,height_cm,weight_kgs,nationality,overall_rating,potential,wage,preferred_foot,international_reputation(1-5),weak_foot(1-5),...,LCB,CB,RCB,RB,value,position_first,position_second,position_third,upper_work_rate,down_work_rate
0,21,1.8288,74.8,146,67,76,0.004405,1,1,3,...,60.2,60.2,60.2,63.2,1200000,4,3,10,3,3


In [18]:
dataframe['num_of_work_rate']=dataframe['upper_work_rate']/dataframe['down_work_rate']
dataframe['num_of_work_rate'].head()

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: num_of_work_rate, dtype: float64

In [19]:
dataframe= dataframe.drop(['work_rate'],axis=1)

In [20]:
for col in dataframe.columns:
    if dataframe[col].isnull:
        
        dataframe[col].fillna(dataframe[col].mean(),inplace=True)

In [21]:
dataframe.head()

,age,height_cm,weight_kgs,nationality,overall_rating,potential,wage,preferred_foot,international_reputation(1-5),weak_foot(1-5),...,CB,RCB,RB,value,position_first,position_second,position_third,upper_work_rate,down_work_rate,num_of_work_rate
0,21,1.8288,74.8,146,67,76,0.004405,1,1,3,...,60.2,60.2,63.2,1200000,4,3,10,3,3,1.0
1,18,1.5494,67.1,76,60,82,0.000000,1,1,3,...,58.2,58.2,59.2,400000,2,5,0,3,3,1.0
2,28,1.7526,64.9,129,63,63,0.000000,1,1,3,...,41.2,41.2,50.2,375000,0,11,5,3,3,1.0
3,20,1.8542,74.8,28,51,61,0.002203,1,1,3,...,48.2,48.2,49.2,60000,11,0,0,3,3,1.0
4,25,1.8288,77.1,27,63,68,0.000000,1,1,3,...,37.2,37.2,38.2,500000,14,0,0,3,3,1.0


In [22]:
data=dataframe[['overall_rating', 'potential', 'wage', 'international_reputation(1-5)',
       'release_clause_euro', 'club_rating', 'reactions', 'value']]

In [23]:
poly=joblib.load('model/poly_data_to2')
xpoly=poly.transform(data.drop(['value',],axis=1))

poly.fit(xpoly,data['value'])


PolynomialFeatures()

In [24]:
model= joblib.load('model/model_regression')

In [25]:
model.score(xpoly,data['value'])

0.9822461777666424

In [26]:
model.predict(xpoly)

array([1361665.02799296,  479490.05202281,  296274.61360919, ...,
        827199.08989565,  639565.36722866, 2394365.50933824])

In [27]:
from sklearn.metrics import mean_squared_error
ypred=model.predict(dataframe.drop(['value'],axis=1))
mean_squared_error(dataframe['value'],ypred)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 36 is different from 83)